# 实训小组作业：量子点目标检测

### 目标

对图像中的量子点进行检测，并计算出其中的量子点数量。(计数)


### 数据

用labelme对数据（包括图像和视频）标记位置，得到数据集

**labelme教程**

labelme是一款开源的标注工具，由python写成

    - 安装

In [1]:
!pip install labelme


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: pip install --upgrade pip


    - 启动

In [3]:
!labelme

2023-05-18 00:53:46.588 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:53:51.666 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:53:54.375 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:55:01.731 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:55:15.767 python[62036:1458531] TSM AdjustCapsLockLEDForKeyTransitionHandling - _ISSetPhysicalKeyboardCapsLockLED Inhibit
2023-05-18 00:56:28.881 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:56:34.267 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:56:36.166 python[62036:1458531] +[CATransaction synchronize] called within transaction
2023-05-18 00:56:53.903 python[62036:1458531] +[CATransaction synchronize] called within transaction


### 原理

在样品中加入荧光物，荧光物会和特定的分子产生结合。

结合后，在激光的激发下，会发出闪烁的荧光。

可以用于测试样品中，是否存在某种特殊的分子。

然而，由于闪烁频率较快，相机采样的频率不对齐，有可能会发生拍照的瞬间荧光正好没有激发的情况(也可能亮，或者不够亮的状态)。

通常是对一个样品采集若干张图像，生成一个图像序列。

所以数据中存在单张图像(由专业人士选择的效果较好的图像)，以及连续多帧拍摄的视频。

我们的目的在于，从数据中查找出发光点的位置，并对齐进行计数。

### 评价标准

**基本知识**

真假与阴阳

- True Positive(**TP**): 实际为正例，预测为正例
- False Positive(**FP**): 实际为负例，预测为正例
- True Negative(**TN**): 实际为负例，预测为负例
- False Negative(**FN**): 实际为正例，预测为负例

如何计算TP和FP？

* 给定ground truth的目标框，和一个预测的目标框，计算二者的重合度(IoU)
* 如果重合度大于一定的阈值（如0.5）, 则视为是一个TP，否则视为一个FP
* 如果重合度小于设定阈值，则视为一个FN
* 如果一个目标被预测了多个框，则将满足重合度要求的，置信度最高的预测框视为TP，其余皆视为FP

TP和FP能计算什么？

* 精度(precision)

$$P=\frac{TP}{TP+FP}$$

* 召回率(recall)

$$R=\frac{TP}{TP+FN}$$

当IoU阈值从0逐渐变化到1时，

精度和recall会逐渐变化，得到PR曲线

精度和召回率可以计算更全面的评价

* F1 score

$$F_1 = \frac{2PR}{P+R}$$

AP的计算

* AP是PR曲线下的面积，越大说明效果越好。

$$AP=\sum P_i$$

mAP的计算

* 度量多类别下的AP

$$mAP=\frac{AP}{num\_class}$$

准确率

* 准确率表明预测框的准确率，即

$$Acc=\frac{TP}{TP+FP+FN}$$

**计数评价**

* 均方误差

    $$MAE=\frac{1}{n}\sum |pred - gt|$$

### 任务说明

方法不限，可以采用深度方法、不同的检测模型，也可以采用传统方法，甚至可以采用分割、GAN等等技术。

技术上没有任何限制，言之有理，能实现更好的性能为最终目的。



**流程**

1. 数据标注：每组都会分配不同的数据进行标注。标注后全部数据集统一收集后整合给大家用于训练和测试；

2. 代码开发：各组组内讨论算法流程

3. 性能评估：(建议在开发前就完成性能评估代码，以测试模型性能) 需要包括至少：F1 score，PR曲线, AP以及MAE4个基本评价指标

4. 报告撰写：按上次作业流程编写报告。**雷同零容忍**

**时间节点**

5月29日前发布任务，6月22日(端午节)前提交最终报告和代码。

**特殊要求**

事先准备好ppt，视实现程度邀请小组成员进行方法介绍

### 一些提示(后续想到会补充)

* 数据集总数较少，但是可以将样本切割成小块，或者滑动窗口将图像截取成相同尺寸的大小，来扩充数据集；

* 但最终评估结果是在完整的图像上进行的，也就是意味着要是切割成小块训练模型，你最后需要将结果拼起来统一度量

* 目标非常小，直接使用faster rcnn等模型，还有yolo这种，会效果不好

* 同时背景非常嘈杂，可以适当采用预处理步骤实现图像增强。